<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/period-and-revolution-time-rigid-pendulum-Vinnie369/blob/main/period_and_revolution_time_rigid_pendulum_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extracting the period of the rigid pendulum

### Pseudocode
1. **Import needed modules:** `numpy` for special functions and linear algebra; `matplotlib.pyplot` for plotting; `scipy.integrate` for `solve_ivp` for finding the pendulum trajectory, and `quad` for integration.
2. **Define ODE function and the integrand:** [add details]
3. **Create utilities:** some basic plotting functions...
4. **Extract the period using the integral:** ...
5. **Extract the period using the differential equation:**...
6. **Plot both results and compare:**...

In [3]:
# import modules
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, quad

### Define ODE function and the integrand

1. `rigid_pendulum_ode(t, y, omega_0=1)` gives the differential equation for the rigid pendulum in the form needed for `solve_ivp`.
2. `period_integrand(theta, theta_0, omega_0=1)` gives the integrand in a form that can used with `quad`.

In [4]:
# define the rigid pendulum ODE and the integrand for integration with quad

# rigid pendulum ode
def rigid_pendulum_ode(t, y, omega_0=1.0):
    """y = [theta, ang_vel]; theta'' + omega_0^2 sin(theta) = 0"""
    theta, ang_vel = y
    return [ang_vel, -(omega_0**2) * np.sin(theta)]

# energy per unit inertia e = 1/2 v^2 + ω0^2 (1 - cos θ)
def energy_per_I(theta, ang_vel, omega_0=1.0):
    return 0.5 * ang_vel**2 + (omega_0**2) * (1 - np.cos(theta))

# convenience: separatrix energy (θ at the top, π)
def separatrix_energy(omega_0=1.0):
    return 2.0 * (omega_0**2)  # e_sep


### Create utilities

1. `basic_plot(x, y, xlabel=None, ylabel=None, figsize=(3,3))` creates a simple plot for fast evaluation of data, visually.
2. `ang_vel_zero_event(t, y, omega_0=1)` creates an event for `solve_ivp` (`ang_vel=0`).
3. `extract_period_int(n_points, eps=1e-3, omega_0=1)` extracts the period on a mesh of `theta_0` using the integral method.
4. `extract_period_ode(n_points, eps=1e-3, omega_0=1)` extracts the perido with `solve_ivp` events.

In [5]:
# create utilities

# basic plotting
def basic_plot(x, y, xlabel=None, ylabel=None, figsize=(3,3)):
  """
  """
  plt.figure(figsize=figsize)
  plt.plot(x, y)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  return plt.show()

# define the event function: ang_vel = 0
def ang_vel_zero_event(t, y, omega_0=1):
  """
  """
  theta, ang_vel = y # unpack state variable
  return ang_vel # set to find values where ang_vel = 0

# add required attributes to the event function
ang_vel_zero_event.terminal = False # keep integrating after event(s) are found
ang_vel_zero_event.direction = -1 # track zeros when going from positive to negative


# extract period on mesh of theta_0 values
def extract_period_int(n_points, eps=1e-3, omega_0=1):
  """
  You should fill this out with the details...
  """

  # extract period with integral on a list of initial angles
  theta_0_list = np.linspace(eps, np.pi - eps, n_points) # list of initial angles

  period_list_int = [] # initialize list of periods

  for theta_0 in theta_0_list:
    period, err = quad(period_integrand, 0, theta_0, args=(theta_0, omega_0))
    period_list_int.append(period)

  period_list_int = np.array(period_list_int) # convert to a numpy array

  period_list_int = np.concatenate([period_list_int[::-1], period_list_int]) # enforcing symmetry to extend data
  theta_0_list = np.concatenate([-theta_0_list[::-1], theta_0_list]) # enforcing symmetry to extend data
  return theta_0_list, period_list_int


# extract period with solve_ivp evetns
def extract_period_ode(n_points, eps=1e-3, omega_0=1):
  """
  Extract period with solve_ivp events on a list of initial angles
  You should fill this out with the details...
  """
  theta_0_list = np.linspace(eps, np.pi - eps, n_points) # list of initial angles

  # range of times for integrating
  t_min = 0
  t_max = 100
  t_span = [t_min, t_max] # time span for solve_ivp

  period_list_ode = [] # initialize list of periods
  for theta_0 in theta_0_list:
    # define state variable for run
    y_0 = [theta_0, 0] # initial condition

    # solve differential equation
    sol = solve_ivp(rigid_pendulum_ode, t_span, y_0, args=(omega_0,),
                    max_step = 0.01,
                    dense_output=False,
                    events=ang_vel_zero_event
                    )
    # output the extracted period
    period = np.mean(np.diff(sol.t_events)) # define period
    period_list_ode.append(period) # append period

  period_list_ode = np.array(period_list_ode) # convert to a numpy array

  period_list_ode = np.concatenate([period_list_ode[::-1], period_list_ode]) # enforcing symmetry to extend data
  theta_0_list = np.concatenate([-theta_0_list[::-1], theta_0_list]) # enforcing symmetry to extend data
  return theta_0_list, period_list_ode

# ...other utlities...

### Testing the integration approach

In [6]:
# # test the integral
# theta_0 = np.pi/2 # initial angle
# omega_0 = 1 # natural frequency
# period, err = quad(period_integrand, 0, theta_0, args=(theta_0, omega_0))

# print(period / (2*np.pi))

In [7]:
# n_points = 50 # number of initial angles
# theta_0_list, period_list_int = extract_period_int(n_points) # extract period on mesh of theta_0
# basic_plot(theta_0_list, period_list_int, xlabel='theta_0', ylabel='period') # plot

### Test the `solve_ivp` approach

In [8]:
# # testing the solve_ivp + event approach

# # range of times for integrating
# t_min = 0
# t_max = 100
# t_span = [t_min, t_max] # time span for solve_ivp

# # state variable
# theta_0 = np.pi - 0.001 # initial angle
# y_0 = [theta_0, 0] # initial condition

# # parameters
# omega_0 = 1 # natural frequency

# # solve differential equation
# sol = solve_ivp(rigid_pendulum_ode, t_span, y_0, args=(omega_0,),
#                 max_step = 0.01,
#                 dense_output=True,
#                 events=ang_vel_zero_event
#                 )

# # output the extracted period
# period = np.mean(np.diff(sol.t_events))
# print(period)

# # quick plot
# t_plot = np.linspace(t_min, t_max, 1000)
# theta_plot = sol.sol(t_plot)[0]
# ang_vel_plot = sol.sol(t_plot)[1]
# basic_plot(t_plot, theta_plot, xlabel='t', ylabel='theta', figsize=(12,2))
# basic_plot(t_plot, ang_vel_plot, xlabel='t', ylabel='ang_vel', figsize=(12,2))

In [9]:
# n_points = 20 # number of theta_0 values
# theta_0_list_ode, period_list_ode = extract_period_ode(n_points)

In [10]:
# basic_plot(theta_0_list_ode, period_list_ode, xlabel='theta_0', ylabel='period',figsize=(3,3))

### Compare the period extracted in the two different ways

In [11]:
# extract period with the integral approach
n_points_int = 100 # number of initial angles
theta_0_list_int, period_list_int = extract_period_int(n_points_int) # extract period on mesh of theta_0

# extract the period with the aolve_ivp approach
n_points_ode = 20 # number of theta_0 values
theta_0_list_ode, period_list_ode = extract_period_ode(n_points_ode) # extract period on mesh of theta_0

NameError: name 'period_integrand' is not defined

In [ ]:
# plot them together
plt.figure(figsize=(5,3))
plt.plot(theta_0_list_int, period_list_int, label='integral')
plt.plot(theta_0_list_ode, period_list_ode, '.', label='solve_ivp')
plt.xlabel('$\\theta_0$')
plt.ylabel('$T$')
plt.xlim(-np.pi-0.2, np.pi+0.2)
plt.ylim(0, 40)
plt.legend()
plt.show()